<div class="alert alert-success">
<font color='black'>    
<br><b>Цели исследования:</b>
<br>Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены. Для выполнения вышепоставленной задачи необходимо:
<br>- подготовить исследование рынка Москвы;
<br>- найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.</font>
</div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Загрузка данных и изучение общей информации

In [ ]:
df = pd.read_csv('/datasets/moscow_places.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print('В датасете представлено заведений - ', df.name.nunique())

In [ ]:
df['category'].value_counts()

<div class="alert alert-success">
<font color='black'>    
<b><br>- В исходном датасете 8405 строк, число уникальных заведений - 5614, тип данных соответствует значениям, изменять название столбцов не требуется.</b></font>
</div>

### Выполнение предобработки данных

In [ ]:
df.duplicated(subset=['name', 'address']).sum()

In [ ]:
df.isna().sum()

<br>- Имеются ропуски в столбцах hours, price, avg_bill, middle_avg_bill, middle_coffee_cup, seats. Удаление пропусков приведет к искажению в исследовании. Далее рассмотрим возможный вариант заполнения пропусков в данных.

In [ ]:
df['name'] = df['name'].str.lower()
df['address'] = df['address'].str.lower()
df['avg_bill'] = df['avg_bill'].str.lower()

In [ ]:
df.duplicated(subset=['name', 'address']).sum()#выявим неявные дубликаты после приведения к нижнему регистру

In [ ]:
df.loc[df.duplicated(subset=['name', 'address'])]#выявим неявные дубликаты после приведения к нижнему регистру

In [ ]:
df.duplicated(subset=['name', 'address']).sum()
print(len(df))
df = df.drop_duplicates(subset=['name', 'address'])#произведем удаление дубликатов
print(len(df))

In [ ]:
new_avg_bill = df.groupby('avg_bill')['middle_avg_bill'].max().dropna()
#Сгруппируем все значения начинающиеся с "средний счёт:". создадим переменную -new_avg_bill 
print(new_avg_bill.index)
print(new_avg_bill.head())

In [ ]:
def func_avg(avg_bill):#Создадим функцию с помощью функции заменим пропуски по совпадению из списка
    if avg_bill in new_avg_bill:
        return new_avg_bill[avg_bill]

In [ ]:
df.loc[df['middle_avg_bill'].isna(), 'middle_avg_bill'] = df['avg_bill'].apply(func_avg)

In [ ]:
df1=df[df['middle_avg_bill'].isna()]
df1=df1[df1['avg_bill'].notnull()]
df1=df1[['avg_bill','middle_avg_bill']]
df1['avg_bill'].unique()

Из полученных результатов, можем сказать что пропуски вызваны из-за разных категорий заведений. Данные из avg_bill распределяются по разным столбцам и это не является ошибкой.

Также считаю, что заполнять средними или медианными значениями по названию заведения остальные столбцы некорректно, так как количество мест в сетевых заведениях, в разных районах, может значительно отличаться. Также в мелких сетях, по условию, могут быть ошибки.
Оставим пропуски как есть.

In [ ]:
df['seats'].describe()

In [ ]:
sns.set_palette('rocket')
sns.boxplot(y='seats', x='category', data=df, hue="category", dodge=False)
sns.set(rc={'figure.figsize':(13,10)})
plt.title('Количество сидячих мест в заведениях')
plt.xlabel('Тип заведения')
plt.ylabel('Количество сидячих мест')
plt.show()

На графике видим выбросы. Скорее всего они связаны с ошибкой при заполнении данных (возможно приписан лишний 0). Оставим все, что меньше 600, посчитаем сколько таких заведений.

In [ ]:
df[df['seats'] > 600]['name'].count()

In [ ]:
(df[df['seats'] > 600]['name'].count()/df['seats'].count())*100

Таких заведений чуть более 1%, можно их удалить

In [ ]:
df600 = df[df['seats'] > 600]
df = df.query('index not in @df600.index')#Сделаем срез данных, исключим заведения с местами более 600

In [ ]:
print('Заведений с числом мест больше 600:', df[df['seats'] > 600]['name'].count())#Проверка результата

In [ ]:
df['street'] = df['address'].str.split(', ').str[1]#Создадим столбец street с названиями улиц из столбца с адресом

In [ ]:
df.head()

In [ ]:
df['is_24/7'] = df['hours'].str.contains('ежедневно, круглосуточно')
#Создадим столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7): логическое значение True — если заведение работает ежедневно и круглосуточно; логическое значение False — в противоположном случае.
df.head()

<div class="alert alert-success">
<font color='black'>    
<br><b>-Имеются пропуски в столбцах hours, price, avg_bill, middle_avg_bill, middle_coffee_cup, seats. Удаление пропусков приведет к искажению в исследовании. Из полученных результатов, можно сделать вывод, что пропуски вызваны из-за разных категорий заведений. Данные из avg_bill распределяются по разным столбцам и это не является ошибкой. Также считаю, что заполнять средними или медианными значениями по названию заведения остальные столбцы некорректно, так как количество мест в сетевых заведениях, в разных районах, может значительно отличаться. Оставим пропуски как есть.
<br> -Обнаружены выбросы в столбце seats. Принято решение оставить данные с количеством посадочных мест меньше 600.</b></font>
</div>

### Анализ данных

#### Какие категории заведений представлены в данных? Исследуйте количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте визуализации. Ответьте на вопрос о распределении заведений по категориям.

In [ ]:
category_name = df.groupby('category')['name'].count().reset_index()
category_name.columns = ['category', 'count']
category_name

In [ ]:
go.Figure(data=[go.Pie(labels=category_name['category'], values=category_name['count'])], 
          layout=go.Layout(title=go.layout.Title(text="Круговая диаграмма категорий заведений")))

In [ ]:
fig = px.bar(category_name, 
             x='category', 
             y='count', 
             text='count',
             title='Количество объектов общественного питания по видам'
            )
fig.update_layout(xaxis_title='Категории заведений',
                  yaxis_title='Количество заведений',
                  xaxis={'categoryorder':'total descending'})
fig.show() # выводим график

<div class="alert alert-success">
<font color='black'>    
<br><b>Можно сделать промежуточный вывод:</b>
<br>По количеству заведений лидируют кафе с численностью 2364, немного меньше ресторан - 2030. На последнем месте булочная - 255.
<br>Кафе и рестораны составляют почти 52,6% от всех заведений. Кофейни, бары/пабы и пиццерии составляют 33,43% от общего числа заведений. На булочные, столовые и быстрое питание приходится 13,97%.</font>
</div>

#### Исследуйте количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте визуализации. Проанализируйте результаты и сделайте выводы.

In [ ]:
df.groupby('category')['seats'].describe().round(2).reset_index()

In [ ]:
mean_seats = df[df['seats'] != 0].groupby('category')['seats'].median().reset_index().sort_values(by='seats', ascending=False).round(2)

f, ax = plt.subplots(figsize=(17, 7))

ax = sns.barplot(data=mean_seats, x='category', y='seats')

ax.set_xlabel('Тип объекта')
ax.set_ylabel('Количество мест')
ax.set_title('Среднее количество мест в заведениях Москвы')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(17, 15))
ax = sns.boxplot(data=df, y='seats', x='category', palette="Set3")
plt.title('Распределения количества посадочных мест по типам заведений')
plt.xlabel('Количество мест')
plt.ylabel('Тип заведений')
plt.show()

<div class="alert alert-success">
<font color='black'>    
<br><b>Больше всего посадочных мест в ресторанах, далее идут заведения быстрого питания, т.к. они больше всего распространены в Москве. Кафе и столовые располагают меньшим количеством мест.</b></font></div>

#### Рассмотрите и изобразите соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?

In [ ]:
chain = df['chain'].value_counts().reset_index()
chain.columns = ['chain', 'count']
chain

In [ ]:
fig = go.Figure(data=[go.Pie(labels=['несетевые', 'сетевые'], values=chain['count'])], layout=go.Layout(
        title=go.layout.Title(text="Соотношение сетевых и несетевых заведений")))
fig.show()

<div class="alert alert-success">
<font color='black'>    
<br><b>Анализируя график можно сделать однозначный вывод, что в Москве почти 62% заведений не являются сетевыми.</b></font></div>

#### Какие категории заведений чаще являются сетевыми? Исследуйте данные и ответьте на вопрос графиком.

In [ ]:
objects = df.groupby(['category', 'chain'])['name'].count().reset_index().sort_values(['category','name','chain'])
objects.columns = ['category', 'chain', 'count']
objects

In [ ]:
fig = px.bar(objects, # загружаем данные и заново их сортируем
             y='count', # указываем столбец с данными для оси X
             x='category', # указываем столбец с данными для оси Y
             text='count', # добавляем аргумент, который отобразит текст с информацией                  
             category_orders={"chain": ["сетевой", "несетевой"]},
             color='chain'
            )
# оформляем график
fig.update_layout(title='Соотношение сетевых заведений',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название категорий',
                 )
fig.show() # выводим график

In [ ]:
chain_t = df.pivot_table(index = 'category', values='name', columns = 'chain', aggfunc='count').reset_index()
chain_t['total'] = chain_t[0] + chain_t[1]
chain_t['rate'] = round((chain_t[1] / chain_t['total'])*100, 0)
chain_t = chain_t.sort_values(by='rate', ascending=False)
chain_t

<div class="alert alert-success">
<font color='black'>    
<br><b>Анализируя график можно сделать вывод, что что большинство за несетевыми заведениями за исключением: кофейня - количество сетевых немного больше. 713/688 или 51 % от общего количества заведений данной категории, пиццерия - сетевых заведений больше. 328/301 или 52 % от общего количества заведений данной категории, булочная - сетевых заметно больше. 156/99 или 61 % от общего количества заведений данной категории</b></font></div>

#### Сгруппируйте данные по названиям заведений и найдите топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. Постройте подходящую для такой информации визуализацию. Знакомы ли вам эти сети? Есть ли какой-то признак, который их объединяет? К какой категории заведений они относятся?

In [ ]:
top_15 = df[df['chain'] == 1].groupby('name').agg({'category' : pd.Series.mode,'district' : 'count'}).sort_values('district', ascending = False).reset_index().head(15)
top_15

In [ ]:
fig = px.bar(top_15, # загружаем данные и заново их сортируем
             x='district', # указываем столбец с данными для оси X
             y='name', # указываем столбец с данными для оси Y
             text='district', # добавляем аргумент, который отобразит текст с информацией
             color='name'                   # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='ТОП-15 популярных сетей в Москве',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название заведений',
                   showlegend=False)
fig.show() # выводим график

<div class="alert alert-success">
<font color='black'>    
<br><b>Из графика видно, что самая популярная сеть в Москве - шоколадница. На втором и третьем месте пиццерии: доминос пицца и додо пицца. Меньше всего заведений в сети му-му.</b></font></div>

#### Какие административные районы Москвы присутствуют в датасете? Отобразите общее количество заведений и количество заведений каждой категории по районам. Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
district_chain = df.groupby(['district', 'category', 'name']).agg({'rating' : 'median', 'address' : 'count'})
district_chain = district_chain.sort_values('rating', ascending = False).reset_index()
district_chain = district_chain.rename(columns={'address':'count'})
district_chain = district_chain[district_chain['name'].isin(top_15['name'])]
district_chain.head()

In [ ]:
fig = px.bar(district_chain, # загружаем данные и заново их сортируем
             x='count', # указываем столбец с данными для оси X
             y='district', # указываем столбец с данными для оси Y                              
             color='category'
            )
# оформляем график
fig.update_layout(title='Количество заведений каждой категории по районам',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название района',
                   yaxis={'categoryorder':'total ascending'}
                 )
fig.show() # выводим график

<div class="alert alert-success">
<font color='black'>    
<br><b>В датасете представлено всего 9 административных районов Москвы.
По графику видно, что больше всего заведений по всем категориям находится в Центральном административном округе. Больше всего в нем категорий кафе, кофейня и ресторан. Отчетливо прослеживается что категория заведений кафейня хорошо распределена по всем округам. Столовых меньше всего по всем округам. У ресторанов, пиццерий и кафе примерно одинаковое количество. Наименьшее количетсво заведений представлено в Северо-Западном администраивном округе. Категории быстрого питания наименее представлена в Центральном административном округе и более распространена в остальных округах.
</b></font></div>

#### Визуализируйте распределение средних рейтингов по категориям заведений. Сильно ли различаются усреднённые рейтинги в разных типах общепита?

In [ ]:
rating = df.groupby('category').agg({'rating' : 'mean'}).round(1).sort_values('rating', ascending = False).reset_index()
rating

In [ ]:
fig = px.bar(rating, # загружаем данные и заново их сортируем
             x='rating', # указываем столбец с данными для оси X
             y='category', # указываем столбец с данными для оси Y
             text='rating', # добавляем аргумент, который отобразит текст с информацией
             color='category'                   
            )
# оформляем график
fig.update_layout(title='Распределение средних рейтингов по категориям заведений',
                   xaxis_title='Рейтинг',
                   yaxis_title='Название категорий')
fig.update_xaxes(range=[4, 4.5])
fig.show() # выводим график

<div class="alert alert-success">
<font color='black'>    
<br><b>Анализируя график можно сделать вывод, что у категорий бар,паб самый высокий рейтинг 4.4. У пиццерий, ресторанов, кофеин и булочных примерно одинаковый рейтинг. Наименьший рейтинг у заведений быстрого питания.
</b></font></div>

#### Постройте фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете

In [ ]:
rating_1 = df.groupby('district', as_index=False)['rating'].agg('mean').round(1)
rating_1

In [ ]:
import json# подключаем модуль для работы с JSON-форматом
with open('/datasets/admin_level_geomap.geojson', 'r') as f:
    geo_json = json.load(f)# читаем файл и сохраняем в переменной

In [ ]:
# импортируем карту и хороплет
from folium import Map, Choropleth

# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
a = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_1,
    columns=['district', 'rating'],
    key_on='feature.name',
    legend_name='Средний рейтинг заведений по районам',
).add_to(a)

# выводим карту
a

<div class="alert alert-success">
<font color='black'>    
<br><b>По карте видно, что самый высокий рейтинг в заведениях в Центральном админинстративном округе - 4.4. Самый низкий - в Юго-Восточном админинстратвном и Северо-Восточный административных округах- 4.1.
</b></font></div>

#### Отобразите все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

In [ ]:
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
a = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(a)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(clusters, axis=1)

# выводим карту
a

<div class="alert alert-success">
<font color='black'>    
<br><b>Анализируя карту можно сделать вывод, что основная масса заведений сконцентрирована в центре города. Наименьшее количество заведений на севере и на юге города.</b></font></div>

#### Найдите топ-15 улиц по количеству заведений. Постройте график распределения количества заведений и их категорий по этим улицам. Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
df_streets = df[df['street'].notnull()]
df_streets_top15 = df_streets['street'].value_counts().reset_index().head(15)
df_streets_top15.columns = ['street_name', 'count']
df_streets_top15

In [ ]:
#создадим таблицу с названиями улиц и категорий
category_street = df.groupby(['street', 'category'])['name'].count().reset_index().sort_values('name', ascending=False)
category_street.columns = ['street_name', 'category', 'count']
category_street.head(10)

In [ ]:
#оставим только улицы из топ 15
category_street_top15 = category_street[category_street['street_name'].isin(df_streets_top15['street_name'])]
category_street_top15.head()

In [ ]:
fig = px.bar(category_street_top15, # загружаем данные и заново их сортируем
             x='count', # указываем столбец с данными для оси X
             y='street_name', # указываем столбец с данными для оси Y
             #text='district', # добавляем аргумент, который отобразит текст с информацией
             #barmode='group',                   
             color='category'
            )
# оформляем график
fig.update_layout(title='Количество заведений каждой категории по районам',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название улиц',
                   yaxis={'categoryorder':'total ascending'}
                   #font=dict(size=8),
                   #autosize=False,
                   #width=1000,
                   #height=800
                 )
fig.show() # выводим график

<div class="alert alert-success">
<font color='black'>    
<br><b>Из данных графика можно сделать вывод, что больше всего заведений расположено на проспекте Мира. Часто встречаются категории кафе, кофейни и рестораны. Столовых и булочных меньше всего. Далее по количеству заведений идет улица Профсоюзная. Меньше всего заведений на улице Миклухо-Маклая. Топ-3 популярных категорий кафе, кофейни и рестораны.</b></font></div>

#### Найдите улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?

In [ ]:
one = df['street'].value_counts().reset_index()
one.columns = ['street_name', 'cafe_count']
one = one[one['cafe_count'] == 1]
one.head()

In [ ]:
print('Итого улиц, на которых находится только один объект общепита -', len(one))

In [ ]:
#добавим районы
str_category = category_street[category_street['street_name'].isin(one['street_name'])]
str_category = str_category.groupby('category')['street_name'].count().sort_values(ascending=False)
str_category

<div class="alert alert-success">
<font color='black'>    
<br><b>Из сформированных таблиц видно, что 457 улиц имеют только одно заведение. Больше всего из них относятся к категории кафе - 159 заведений.</b></font></div>

#### Значения средних чеков заведений хранятся в столбце middle_avg_bill. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитайте медиану этого столбца для каждого района. Используйте это значение в качестве ценового индикатора района. Постройте фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируйте цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?

In [ ]:
median_avg_bill = df.groupby('district')['middle_avg_bill'].median().reset_index()
median_avg_bill

In [ ]:
# создаём карту Москвы
b = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=median_avg_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    legend_name='Средний чек заведений по районам',
).add_to(b)

# выводим карту
b

<div class="alert alert-success">
<font color='black'>    
<br><b>По интерактивной карте видно, что самый высокий средний чек в Центральном и Западном административных округах. Самый маленький в Юго-Восточном  и Северо-Восточном административных округах. Средний чек округов вокруг центрального в среднем отличается практически в 2 раза.</b></font></div>

<div class="alert alert-success">
<font color='black'>    
<br><b>ОБЩИЙ ВЫВОД ИССЛЕДОВАНИЯ:</b>
<br> - По результатам исследования известно, что по количеству заведений лидируют кафе с численностью 2366, немного меньше ресторан - 2031. На последнем месте булочная - 255. Кафе и рестораны составляют почти 52,6% от всех заведений. Кофейни, бары/пабы и пиццерии составляют 33,43% от общего числа заведений. На булочные, столовые и быстрое питание приходится 13,97%.   
<br> - Больше всего посадочных мест в ресторанах, далее идут заведения быстрого питания, т.к. они больше всего распространены в Москве. Кафе и столовые располагают меньшим количеством мест.
<br> - В Москве почти 62% заведений не являются сетевыми.
<br> - Большинство за несетевыми заведениями за исключением: кофейня - количество сетевых немного больше. 713/688, пиццерия - сетевых заведений больше. 328/301, булочная - сетевых заметно больше. 156/99
<br> - Самая популярная сеть в Москве - шоколадница. На втором и третьем месте пиццерии: доминос пицца и додо пицца. Меньше всего заведений в сети му-му.
<br> - Всего представлено  9 административных районов Москвы. Больше всего заведений по всем категориям находится в Центральном административном округе. Больше всего в нем категорий кафе, кофейня и ресторан. Отчетливо прослеживается что категория заведений кафейня хорошо распределена по всем округам. Столовых меньше всего по всем округам. У ресторанов, пиццерий и кафе примерно одинаковое количество. Наименьшее количетсво заведений представлено в Северо-Западном администраивном округе. Категории быстрого питания наименее представлена в Центральном административном округе и более распространена в остальных округах.
<br> -У категорий бар,паб самый высокий рейтинг 4.4. У пиццерий, ресторанов, кофеин и булочных примерно одинаковый рейтинг. Наименьший рейтинг у заведений быстрого питания.
<br> - Самый высокий рейтинг в заведениях в Центральном админинстративном округе - 4.4. Самый низкий - в Юго-Восточном админинстратвном и Северо-Восточный административных округах- 4.1.
<br> -Основная масса заведений сконцентрирована в центре города. Наименьшее количество заведений на севере и на юге города.
<br> -  Из данных графика можно сделать вывод, что больше всего заведений расположено на проспекте Мира. Часто встречаются категории кафе, кофейни и рестораны. Столовых и булочных меньше всего. Далее по количеству заведений идет улица Профсоюзная. Меньше всего заведений на улице Миклухо-Маклая. Топ-3 популярных категорий кафе, кофейни и рестораны.
<br> -457 улиц имеют только одно заведение. Больше всего из них относятся к категории кафе - 159 заведений.
<br> -По интерактивной карте видно, что самый высокий средний чек в Центральном и Западном административных округах. Самый маленький в Юго-Восточном и Северо-Восточном административных округах. Средний чек округов вокруг центрального в среднем отличается практически в 2 раза.</font></div>

### Детализируем исследование: открытие кофейни

#### Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

In [ ]:
coffein = df[df['category'] == 'кофейня']
print('Количество кофеин в датасете -', len(coffein))

In [ ]:
# создаем карту
с = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаем пустой кластер и добавляем его на карту
marker_cluster = MarkerCluster().add_to(с)

# функция, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffein.apply(create_clusters, axis=1)

# выводим карту
с

<div class="alert alert-success">
<font color='black'>    
<br><b>Всего в датасете представлено 1401 заведений в категории кофейня. Большая часть кофеин располагается в Центральном административном округе. Наименьшее число кофеин находится в Восточном и Южной административных округах.</b></font></div>

#### Есть ли круглосуточные кофейни?

In [ ]:
coffein_24_7 = coffein[coffein['is_24/7'] == True]
print('Количество круглосуточных кофеен в городе - ', len(coffein_24_7))

In [ ]:
# создаём карту Москвы
d = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(d)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffein_24_7.apply(create_clusters, axis=1)

# выводим карту
d

<div class="alert alert-success">
<font color='black'>    
<br><b> Количество круглосуточных кофеен в городе - 59, большинство из которых в центре города.</b></font></div>

#### Какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
rating = coffein.groupby('district', as_index=False)['rating'].agg('mean').round(2).sort_values('rating', ascending=False)
rating

In [ ]:
# создаём карту
e = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data = rating,
    columns=['district', 'rating'],
    key_on='feature.name',
    legend_name='Средний рейтинг заведений по районам',
).add_to(e)

# выводим карту
e

<div class="alert alert-success">
<font color='black'>    
<br><b>По карте видно, что самый высокий рейтинг в  Центральном административном и Северо-Западном административных округах. Самый низкий рейтинг у Западного административного округа.</b></font></div>

#### На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
cup_of_cappuccino = coffein.groupby('district', as_index=False)['middle_coffee_cup'].agg('mean').round(0).sort_values('middle_coffee_cup', ascending=False)
print(cup_of_cappuccino)
price = round(cup_of_cappuccino['middle_coffee_cup'].mean(), 0)
print(f'Средняя стоимость чашки:', price)

In [ ]:
g = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=cup_of_cappuccino,
    columns=['district', 'middle_coffee_cup'],
    key_on='feature.name',
    legend_name='Средняя цена чашки кофе по районам',
).add_to(g)

# выводим карту
g

<div class="alert alert-success">
<font color='black'>    
<br><b>Дороже всего чашка кофе в Западном и Центральном административных округах. Дешевле всего в Юго-Восточном административном округе. Средняя стоимость чашки кофе - 171.</b></font></div>

<div class="alert alert-success">
<font color='black'>    
<br><b>Рекомендации по открытию нового заведения:</b>
<br>Исходя из логики что самые популярные категории заведений: кафе, ресторан и кофейни, можно сделать вывод, что открытие заведения из вышеуказанных категорий может оказаться наимболее успешным предприятием;
<br>Териториально для открытия можно рассмотреть Центральный и Западный административные округа. Средний чек здесь наиболее высок по сравнению с остальными;
<br>Можно рассмотреть открытие заведения на одной из топ-15 улиц по количеству заведений, такие как  - Проект Мира. Высокая конкуренция сопровождается так же высоким траффиком.
<br>По количетсву посадочных мест стоит ориентироваться на цифру до 100 мест;
<br>Средняя цена на капучино должна быть от 170 до 175 рублей, что попадает в диапазон средних значений по городу.</font></div>

<b>Презинтация: https://disk.yandex.ru/d/kD48ygZ9jcfHIw</b>